<a href="https://colab.research.google.com/github/AhmadHossain8/Food-Image-Classification/blob/main/Inception_V3_Normal_Training_With_Parameter_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
from torch.utils.data import Dataset
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
import os
import time
import pandas as pd
from PIL import Image
from torchvision import transforms,models
import matplotlib.pyplot as plt
from skimage import io,transform

device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

"""**Unzip**"""

DATA_SET_ZIP = '/content/drive/MyDrive/Food 11.zip'
with zipfile.ZipFile(DATA_SET_ZIP,"r") as zip_ref:
  zip_ref.extractall("data")




In [ ]:
path_of_training_data = '/content/data/training'
path_of_evaluation_data = '/content/data/evaluation'
path_of_validation_data = '/content/data/validation'


In [ ]:
class Food_11_Dataset(Dataset):
  def __init__(self,data_dir, transform=None):
    self.date_dir = data_dir
    self.image_list = os.listdir(data_dir)
    self.transform = transform

  def __getitem__(self,idx):
    location = os.path.join(self.date_dir,self.image_list[idx])
    image = Image.open(location)
    if self.transform is not None:
            image = self.transform(image)
    label = self.image_list[idx]
    label = label.split("_")
    label = int(label[0])
    return image,label

  def __len__(self):
    return len(self.image_list)

In [ ]:
batch_size = 64

transform1 = transforms.Compose(
    [transforms.Resize((299,299)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])

transform2 = transforms.Compose(
    [transforms.Resize((299,299)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.5548, 0.4503, 0.3438],[0.2281, 0.2386, 0.2374])
    ])

trainset1 = Food_11_Dataset(path_of_training_data,transform=transform1)
evaluationset1 = Food_11_Dataset(path_of_evaluation_data,transform=transform1)
validset1 = Food_11_Dataset(path_of_validation_data,transform=transform1)


"""**DataLoader**"""

#for normalization1
trainset_loader1 = torch.utils.data.DataLoader(trainset1, batch_size=batch_size, shuffle=True)
evaluationset_loader1 = torch.utils.data.DataLoader(evaluationset1, batch_size=batch_size, shuffle=True)
validset_loader1 = torch.utils.data.DataLoader(validset1, batch_size=batch_size, shuffle=True)


import torch.nn as nn
number_of_class=11
import math

#model parameter average
model1 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
model1.fc = torch.nn.Linear(2048, 11)
model1.aux_logits = False 
model1.to(device)

import copy
num_epochs = 100

#for normalization1
learning_rate1 = 0.001



PATH1 = "/content/drive/MyDrive/New Test/model1.pth"
AVG_PATH1 = "/content/drive/MyDrive/New Test/my_saves_avg.pth"


#for normalization1
criterion1 = nn.CrossEntropyLoss()
optimizer1 = torch.optim.SGD(model1.parameters(), lr=learning_rate1, weight_decay = 0.005, momentum = 0.9)


epoch1 = 0
loss1 = 0.0
best_acc1 = 0.0
best_model_weights1 = []
avg_model_weights1 = []



if os.path.getsize(PATH1) > 0:
  checkpoint = torch.load(PATH1)
  model1.load_state_dict(checkpoint['model_state_dict'])
  optimizer1.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch1 = checkpoint['epoch']
  loss1 = checkpoint['loss']
  best_acc1 = checkpoint['best_acc']
  best_model_weights1 = copy.deepcopy(model1.state_dict())


#Normal training
model2 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
model2.fc = torch.nn.Linear(2048, 11)
model2.aux_logits = False 
model2.to(device)


PATH2 = "/content/drive/MyDrive/New Test/model2.pth"

#for normalization1
criterion2 = nn.CrossEntropyLoss()
optimizer2 = torch.optim.SGD(model2.parameters(), lr=learning_rate1, weight_decay = 0.005, momentum = 0.9)


epoch2 = 0
loss2 = 0.0
best_acc2 = 0.0
best_model_weights2 = []



if os.path.getsize(PATH2) > 0:
  checkpoint = torch.load(PATH2)
  model2.load_state_dict(checkpoint['model_state_dict'])
  optimizer2.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch2 = checkpoint['epoch']
  loss2 = checkpoint['loss']
  best_acc2 = checkpoint['best_acc']
  best_model_weights2 = copy.deepcopy(model2.state_dict())


total_step1 = len(trainset_loader1)


In [ ]:
len(validset1)

In [ ]:
model1_info = {'train_loss' : [] , 'val_acc' : []}
PATH1_acc = "/content/drive/MyDrive/New Test/model1_acc.pth"
if os.path.getsize(PATH1_acc) > 0:
  model1_info = torch.load(PATH1_acc)

if os.path.getsize(AVG_PATH1) > 0:
  checkpoint = torch.load(AVG_PATH1)
  avg_model_weights1 = checkpoint['model_state_dict']

prev_model = []
if len(avg_model_weights1) != 0:
  prev_model = avg_model_weights1


model2_info = {'train_loss' : [] , 'val_acc' : []}
PATH2_acc = "/content/drive/MyDrive/New Test/model2_acc.pth"
if os.path.getsize(PATH2_acc) > 0:
  model2_info = torch.load(PATH2_acc)

In [ ]:
start_time = time.time()
new_time1 = time.time()
new_time2 = time.time()
"""**Train**"""

for epoch in range(num_epochs):
    new_time1 = time.time()
    start_time = time.time()
    #Model1
    print('Epoch [{}/{}]' .format(epoch+1, num_epochs))
    model1.train()
    for i, (images, labels) in enumerate(trainset_loader1):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        
        # Forward pass
        outputs = model1(images)
        loss1 = criterion1(outputs, labels)
        
        # Backward and optimize
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()

    print ('Model 1 Step [{}/{}], Loss: {:.4f}' 
                   .format( i+1, total_step1, loss1.item()))
    model1_info['train_loss'].append(loss1.item())

    if epoch+1 == 50:
      prev_model = best_model_weights1
       
    elif epoch+1 > 50:
        temp_model = copy.deepcopy(model1.state_dict())
        for key in prev_model:
          prev_model[key] = (prev_model[key] + temp_model[key]) / 2.0
          model1.load_state_dict(prev_model)
    torch.save({
          'model_state_dict': model1.state_dict(),
          },AVG_PATH1)       
       
    # Validation
    model1.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for images, labels in validset_loader1:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        accuracy= 100 * correct / total
        model1_info['val_acc'].append(accuracy)
        torch.save(model1_info,PATH1_acc)  
        print('Accuracy of the network on the {} validation images: {} % and time taken {} min'.format(total, accuracy,(time.time() - new_time1)/60))
        if accuracy > best_acc1 : 
            best_acc1 = (100 * correct / total)
            best_model_weights1 = copy.deepcopy(model1.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model1.state_dict(),
            'optimizer_state_dict': optimizer1.state_dict(),
            'loss': loss1,
            'best_acc' : best_acc1,
            },PATH1)
    #Model 2
    new_time2 = time.time()
    model2.train()
    for i, (images, labels) in enumerate(trainset_loader1):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        
        # Forward pass
        outputs = model2(images)
        loss2 = criterion2(outputs, labels)
        
        # Backward and optimize
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()

    print ('Model 2 Step [{}/{}], Loss: {:.4f}' 
                   .format( i+1, total_step1, loss2.item()))
    model2_info['train_loss'].append(loss2.item())

    # Validation
    model2.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for images, labels in validset_loader1:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        accuracy= 100 * correct / total
        model2_info['val_acc'].append(accuracy)
        torch.save(model2_info,PATH2_acc)  
        print('Accuracy of the network on the {} validation images: {} % and time taken {} min'.format(total, accuracy,(time.time() - new_time2)/60))
        if accuracy > best_acc2 : 
            best_acc2 = (100 * correct / total)
            best_model_weights2 = copy.deepcopy(model2.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model2.state_dict(),
            'optimizer_state_dict': optimizer2.state_dict(),
            'loss': loss2,
            'best_acc' : best_acc2,
            },PATH2)
  

elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

model1.load_state_dict(best_model_weights1)
model1.eval()

#For Model1
with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for images, labels in validset_loader1:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        accuracy= 100 * correct / total
    
        print('Accuracy of the model1 on the {} validation images: {} %'.format(total, accuracy))

with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for images, labels in evaluationset_loader1:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        accuracy= 100 * correct / total
    
        print('Accuracy of the model1 on the {} Evaluation images: {} %'.format(total, accuracy))


model2.load_state_dict(best_model_weights2)
model2.eval()
with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for images, labels in validset_loader1:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        accuracy= 100 * correct / total
    
        print('Accuracy of the model 2 on the {} validation images: {} %'.format(total, accuracy))

with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for images, labels in evaluationset_loader1:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        accuracy= 100 * correct / total
    
        print('Accuracy of the model 2 on the {} Evaluation images: {} %'.format(total, accuracy))

#MODEL 1
plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 1")
plt.plot(model1_info['val_acc'],label="val")
plt.plot(model1_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()


#MODEL 2
plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 1")
plt.plot(model2_info['val_acc'],label="val")
plt.plot(model2_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()
